### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [2]:
from diffusion.data_loaders.motion_dataset import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_walk.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.847532, 0.998678, 0.01410399999999995, 0.049422999999999995, -0.0006980000000000042, 0.019374995056800275, 0.008037254877450587, -0.09523902811084285, -0.0, 0.0, -0.0, -0.15553532463598624, 0.23919429256424163, 0.20739656997070288, 0.170571, 0.3529631848273465, -0.2610682953696931, -0.24560532144975333, 0.581348, 0.02035205257945668, -0.5175742452141794, -0.11376339039728192, -0.249116, 0.020556236630260034, -0.019534498786735962, 0.0655269812790598, -0.05606350142619236, 0.15209578259548684, 0.1827420948157945, -0.391532, 0.1931167851688944, -0.2978918547932108, -0.08305715225197069] 35


(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.1931, -0.2979, -0.0831],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.1812, -0.2662, -0.1258],
         [ 0.0817, -0.0105,  0.8483,  ...,  0.1293, -0.1646, -0.1874],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ...,  0.0799, -0.2361, -0.1045],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0750, -0.2538, -0.1148],
         [ 1.0080,  0.0138,  0.8721,  ...,  0.0762, -0.2713, -0.1198]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02])}

### Setup Model
Configure your savepaths here, they will all be stored under the logs folder later on

In [3]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-constrained-sampling-holding-a-box"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [4]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 32
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [5]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 100
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 32
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 100
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box/diffusion_config.pkl



### Setup Training

In [6]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box/trainer_config.pkl



### Test if model and training loop works

In [7]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 32, 35])
x.shape torch.Size([1, 35, 32])
x.shape torch.Size([1, 32, 32])
1
xfinal.shape torch.Size([1, 32, 16])
x.shape torch.Size([1, 64, 16])
2
xfinal.shape torch.Size([1, 64, 8])
x.shape torch.Size([1, 128, 8])
3
xfinal.shape torch.Size([1, 128, 4])
x.shape torch.Size([1, 256, 4])
4
xfinal.shape torch.Size([1, 256, 4])
xt1.shape torch.Size([1, 256, 4])
xt2.shape torch.Size([1, 256, 4])
xt3.shape torch.Size([1, 256, 4])
pop.shape torch.Size([1, 256, 4])
pop.shape torch.Size([1, 128, 8])
pop.shape torch.Size([1, 64, 16])


(tensor([[[ 0.3503,  0.3436, -0.2079,  ..., -0.7957, -0.3442,  0.1303],
          [ 0.3560,  0.9618,  0.2141,  ...,  0.4084, -0.0849,  0.3002],
          [-0.0770,  0.2594,  0.4456,  ..., -0.1111, -0.0041, -0.2033],
          ...,
          [ 0.0397,  0.2979, -0.0131,  ..., -0.0317, -0.1129,  0.0970],
          [-0.2649,  0.1902, -0.3499,  ..., -0.6584,  0.1435, -0.1275],
          [ 0.1463,  0.1145,  0.0952,  ..., -0.2088,  0.1733,  0.2614]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 32, 35]))

In [8]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [9]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box/state_0.pt
0:   0.1780 | a0_loss:   0.0477 | t:   0.6152
100:   0.0610 | a0_loss:   0.0203 | t:   8.5217
200:   0.0570 | a0_loss:   0.0194 | t:   8.5486
300:   0.0557 | a0_loss:   0.0191 | t:   8.8263
400:   0.0553 | a0_loss:   0.0190 | t:   8.1618
500:   0.0551 | a0_loss:   0.0189 | t:   7.7842
600:   0.0551 | a0_loss:   0.0189 | t:   7.7909
700:   0.0550 | a0_loss:   0.0189 | t:   7.7804
800:   0.0550 | a0_loss:   0.0189 | t:   7.7183
900:   0.0550 | a0_loss:   0.0189 | t:   7.7646
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box/state_1.pt


### (Optionally) load a checkpoint

In [8]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(1)
model = trainer.ema_model

FileNotFoundError: [Errno 2] No such file or directory: '/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-constrained-sampling-holding-a-box/state_1.pt'

### Sample from model

In [10]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions)


                                                                                                    
1 / 100 [                                                            ]   1% | 2.5 Hz
t : 99 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 100 [#                                                           ]   2% | 4.9 Hz
t : 98 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 100 [#                                                           ]   3% | 7.2 Hz
t : 97 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                       

### Sanity check to check if output is similar to train data

In [11]:
sample, sample.trajectories.shape

(Sample(trajectories=tensor([[[ 9.1174e-04, -9.9102e-04,  8.4632e-01,  ...,  1.8862e-01,
           -3.0393e-01, -8.4677e-02],
          [ 3.9201e-02, -1.0069e-03,  8.4018e-01,  ...,  1.8243e-01,
           -2.6055e-01, -1.0799e-01],
          [ 8.1783e-02, -2.2209e-02,  8.6250e-01,  ...,  1.2992e-01,
           -1.5786e-01, -1.8062e-01],
          ...,
          [ 9.5728e-01,  5.6613e-03,  8.7124e-01,  ...,  7.5783e-02,
           -2.3263e-01, -1.1508e-01],
          [ 9.7419e-01,  1.2902e-02,  8.7215e-01,  ...,  7.7425e-02,
           -2.4291e-01, -1.1937e-01],
          [ 9.7243e-01,  2.6678e-02,  8.5479e-01,  ...,  7.7567e-02,
           -2.5096e-01, -1.1908e-01]]], device='cuda:0'), values=tensor([0.], device='cuda:0'), chains=None),
 torch.Size([1, 32, 35]))

In [12]:
dataset[0], dataset[0].trajectories.shape

(Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.1931, -0.2979, -0.0831],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.1812, -0.2662, -0.1258],
         [ 0.0817, -0.0105,  0.8483,  ...,  0.1293, -0.1646, -0.1874],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ...,  0.0799, -0.2361, -0.1045],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0750, -0.2538, -0.1148],
         [ 1.0080,  0.0138,  0.8721,  ...,  0.0762, -0.2713, -0.1198]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02])}),
 to

### Check if constrained sampling is working

The values at index 13 to 20 should be 0 to look like its carrying a box

In [15]:
print(sample[0][0][0])

tensor([ 9.1174e-04, -9.9102e-04,  8.4632e-01,  1.0030e+00,  3.9378e-03,
         4.8291e-02, -4.4133e-03,  1.9962e-02,  1.0199e-02, -9.2926e-02,
        -9.1129e-03,  2.0096e-03,  9.2612e-03,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.5700e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.5700e+00,  2.2723e-02, -5.1857e-01, -1.0872e-01, -2.5256e-01,
         2.0410e-02, -2.0062e-02,  6.6590e-02, -5.4197e-02,  1.4811e-01,
         1.8691e-01, -3.9414e-01,  1.8862e-01, -3.0393e-01, -8.4677e-02],
       device='cuda:0')


### Save results to logs

In [16]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion {i} saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="motion1.npy")

Motion 0 saved as motion1.npy
